# Imports

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

# Accessing the API

In [2]:
# Load API Credentials
with open('/Users/nicke/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['API Key'], timeout_s=5.0)

In [3]:
# set our API call parameters 
LOCATION = 'Columbus, OH'
TERM = 'sushi'

# Creating the JSON File

In [4]:
# Specifying JSON_FILE filename
JSON_FILE = "Data/results_in_progress_Columbus_sushi.json"
JSON_FILE

'Data/results_in_progress_Columbus_sushi.json'

In [5]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

# Getting Number of Results Per Page

In [6]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_Columbus_sushi.json already exists. Deleting previous file...
[i] Data/results_in_progress_Columbus_sushi.json not found. Saving empty list to new file.
- 0 previous results found.


12

# For Loop to Call Each Page

In [7]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/12 [00:00<?, ?it/s]

# Converting JSON to Dataframe

In [8]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,6Po-H6z_RIChd31MGWdeyQ,the-1126-restaurant-columbus-2,The 1126 Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/U848m4...,False,https://www.yelp.com/biz/the-1126-restaurant-c...,294,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.5,"{'latitude': 39.98538, 'longitude': -83.00478}","[delivery, pickup]",$$$,"{'address1': '1126 North High St', 'address2':...",+16147253435,(614) 725-3435,1231.795205
1,ei7tFGEM4y3r2Jtw5FAyKQ,akai-hana-columbus,Akai Hana,https://s3-media1.fl.yelpcdn.com/bphoto/jExxnx...,False,https://www.yelp.com/biz/akai-hana-columbus?ad...,693,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 40.0508, 'longitude': -83.05175}","[delivery, pickup]",$$,"{'address1': '1173 Old Henderson Rd', 'address...",+16144515411,(614) 451-5411,9181.822565
2,e3geg9-gF_nRkdVFf00ytg,song-lan-restaurant-dublin,Song Lan Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/K3HJYb...,False,https://www.yelp.com/biz/song-lan-restaurant-d...,24,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",3.5,"{'latitude': 40.102551, 'longitude': -83.109778}",[],NaN,"{'address1': '6628 Riverside Dr', 'address2': ...",,,16758.351993
3,Lq6E0bhVn30hKLDyQ5PVIw,wild-ginger-hilliard,Wild Ginger,https://s3-media3.fl.yelpcdn.com/bphoto/zOjRMd...,False,https://www.yelp.com/biz/wild-ginger-hilliard?...,317,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.5,"{'latitude': 39.9864571541548, 'longitude': -8...",[delivery],$$,"{'address1': '1726 Hilliard Rome Rd', 'address...",+16148766888,(614) 876-6888,13658.861493
4,FqnLYv7yxK1_thyYy4-6LA,sushi-com-dublin-3,Sushi.com,https://s3-media3.fl.yelpcdn.com/bphoto/NfcIFP...,False,https://www.yelp.com/biz/sushi-com-dublin-3?ad...,172,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 40.1129142602655, 'longitude': -8...",[delivery],$$,"{'address1': '7178 Muirfield Dr', 'address2': ...",+16143368686,(614) 336-8686,20038.961614


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
224,KOjEWa0sR2M6WcEMeNKy5Q,kroger-food-and-pharmacy-westerville-2,Kroger Food and Pharmacy,https://s3-media3.fl.yelpcdn.com/bphoto/pvFNvz...,False,https://www.yelp.com/biz/kroger-food-and-pharm...,25,"[{'alias': 'drugstores', 'title': 'Drugstores'...",3.0,"{'latitude': 40.084659, 'longitude': -82.897257}",[],$$,"{'address1': '5991 S Sunbury Rd', 'address2': ...",+16148951500,(614) 895-1500,13850.792997
225,d2YdPwMV6rFieYjBb-kQWw,northstar-café-columbus-6,Northstar Café,https://s3-media3.fl.yelpcdn.com/bphoto/JsR2Iw...,False,https://www.yelp.com/biz/northstar-caf%C3%A9-c...,545,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.0,"{'latitude': 40.048550795719, 'longitude': -83...","[delivery, pickup]",$$,"{'address1': '4241 N High St', 'address2': '',...",+16147842233,(614) 784-2233,7751.690254
226,G53lSnf94qekjoVW77CIkg,kroger-food-and-pharmacy-westerville-3,Kroger Food and Pharmacy,https://s3-media1.fl.yelpcdn.com/bphoto/aUoSWB...,False,https://www.yelp.com/biz/kroger-food-and-pharm...,47,"[{'alias': 'drugstores', 'title': 'Drugstores'...",2.5,"{'latitude': 40.10912, 'longitude': -82.9274499}",[],$$,"{'address1': '55 W Schrock Rd', 'address2': No...",+16148908660,(614) 890-8660,14962.211449
227,GTaEl-oGi-slHvrK5GdsHQ,kroger-worthington-2,Kroger,https://s3-media3.fl.yelpcdn.com/bphoto/WC4_YD...,False,https://www.yelp.com/biz/kroger-worthington-2?...,35,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",2.5,"{'latitude': 40.1075175, 'longitude': -83.0201...",[],$$,"{'address1': '60 Worthington Mall', 'address2'...",+16148853889,(614) 885-3889,14262.137878
228,V8pOpotD5wWLROBMNK4rQA,kroger-dublin-3,Kroger,https://s3-media3.fl.yelpcdn.com/bphoto/5S1NOz...,False,https://www.yelp.com/biz/kroger-dublin-3?adjus...,52,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.5,"{'latitude': 40.122138, 'longitude': -83.092636}",[],$$,"{'address1': '7625 Sawmill Rd', 'address2': No...",+16149232333,(614) 923-2333,17765.865391


In [9]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [10]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_Columbus_sushi.csv.gz', compression='gzip',index=False)